In [51]:
suppressMessages(library(survminer))
suppressMessages(library(survival))
suppressMessages(library(forestplot))
suppressMessages(library(ggsci))
suppressMessages(library(dplyr))
suppressMessages(library(adjustedCurves))
suppressMessages(library(tidyverse))
suppressMessages(library(cmprsk))
suppressMessages(library(caret))
library(tidyr)

source("funcs/rfuncs.R")

## 1. Load Dataset
---

In [52]:
data.df <- read.table("data/processed/AL_with_ccp_03.tsv", sep="\t", header=T, row.names='Code.ID')

# Rename columns
column_mapping <- c("LC.Isotype"="Kappa_or_lambda_PCD", 
                    "BU.Cardiac.Stage"="BU__BNP_based__cardiac_staging",
                    "BMPC.%"="Bone_marrow_plasma_cells____"
                   )

data.df <- data.df %>% rename(!!!column_mapping)

# Filter for time from admission >=0
data.df <- data.df[data.df$time_from_admission>=0,]

# Collapse Race
data.df <- data.df %>%
   mutate(Race = ifelse(Race %in% c('American_Indian_Alaska_Native','Multiracial','Native_Hawaiian_Pacific', 'Unknown/other'), 'Other', Race))

# Fix NULL values
data.df <- data.df %>% mutate(BU.Cardiac.Stage = ifelse(BU.Cardiac.Stage=="", NA, BU.Cardiac.Stage))
data.df <- data.df %>% mutate(Primary_organ = ifelse(Primary_organ=="", NA, Primary_organ))
data.df <- data.df %>% mutate(LC.Isotype = ifelse(LC.Isotype=="", NA, LC.Isotype))

# assign cluster to var
data.df$m02q3_cluster_n <- factor(data.df$m02q3_cluster_n, levels=c("Low","Intermediate","High"))
data.df <- data.df[complete.cases(data.df[,"m02q3_cluster_n"]),]
data.df$m02q4_cluster_n <- factor(data.df$m02q4_cluster_n, levels=c("Low","Intermediate-Low","Intermediate","High"))
data.df$fna3_cluster_n <- factor(data.df$fna3_cluster_n, levels=c("Low","Intermediate","High"))
data.df$BU.Cardiac.Stage <- as.factor(data.df$BU.Cardiac.Stage)

# As Factor
data.df$LC.Isotype <- as.factor(data.df$LC.Isotype)
data.df$Sex <- as.factor(data.df$Sex)
data.df$Race <- as.factor(data.df$Race)
data.df$Era <- as.factor(data.df$Era)

In [53]:
tx.df <- read.table("data/processed/tx_list.tsv", sep="\t", header=T, row.names="Code.ID")
data.df <- cbind(data.df[row.names(tx.df),],tx.df)

data.df$dara_received <- as.logical(data.df$dara_received)
data.df$pi_received <- as.logical(data.df$pi_received)
data.df$dara_refractory <- as.logical(data.df$dara_refractory)
data.df$pi_refractory <- as.logical(data.df$pi_refractory)

In [223]:
# Load ESKD Data
eskd.df <- read.table("data/processed/treatment_eskd.tsv", sep="\t", header=T, row.names="Code.ID")
eskd.df <- eskd.df[rownames(data.df),]
eskd.df <- cbind(data.df, eskd.df[,c("RRT.Type","treatment_eskd","time_to_eskd","time_to_eskd_from_admission","CR_event","CR_time")])
eskd.df <- eskd.df[complete.cases(eskd.df$time_to_eskd_from_admission),]

# Filter for whether or not patients had ESKD at baseline
eskd.df <- eskd.df[eskd.df$time_to_eskd_from_admission>0,]

# Set factor orders
eskd.df$Renal.Stage..Palladini. <- factor(eskd.df$Renal.Stage..Palladini., levels=c("Stage I","Stage II","Stage III"))
eskd.df$fna3_cluster_n <- factor(eskd.df$fna3_cluster_n, levels=c("High","Low","Intermediate"))

## 2. All-Cause MOrtality

---

In [219]:
mod.dara_treated <- coxph(Surv(time_from_admission, status) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=data.df[data.df$dara_received==T,], x=TRUE, tt=function(x,t,...) x * log(t))

mod.dara_or_pi_treated <- coxph(Surv(time_from_admission, status) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=data.df[data.df$dara_received==T | data.df$pi_received==T,], x=TRUE, tt=function(x,t,...) x * log(t))

mod.pi_treated <- coxph(Surv(time_from_admission, status) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=data.df[data.df$pi_received==T,], x=TRUE, tt=function(x,t,...) x * log(t))

# First-line
data.filt.df <- data.df[data.df$first_line %in% c("HDM/SCT","Melphalan-based regimen","Proteasome inhibitor-based", "Daratumumab-based","IMiD-based"),]
data.filt.df$first_line <- factor(data.filt.df$first_line, levels=c("HDM/SCT","Melphalan-based regimen","Proteasome inhibitor-based", "Daratumumab-based","IMiD-based")) 

mod.firstline_interaction <- coxph(Surv(time_from_admission, status) ~ fna3_cluster_n*first_line+ Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=data.filt.df, x=TRUE, tt=function(x,t,...) x * log(t))

mod.firstline <- coxph(Surv(time_from_admission, status) ~ fna3_cluster_n + first_line + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=data.filt.df, x=TRUE, tt=function(x,t,...) x * log(t))

Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
"Loglik converged before variable  7 ; coefficient may be infinite. "


In [220]:
coxSummary2 <- function(cox_models, var, filt = TRUE) 
{
    i <- 1
    result_list = list()
    for (i in seq_along(cox_models)) {
        formula <- Reduce(paste, deparse(formula(cox_models[[i]])))
        hazard_ratios <- getCoef(cox_models[[i]])$HR
        names(hazard_ratios) <- rownames(getCoef(cox_models[[i]]))
        if (filt == T) {
            hazard_ratios <- hazard_ratios[names(hazard_ratios)[grepl(paste0("^", 
                var), names(hazard_ratios))]]
            names(hazard_ratios) <- sub(paste0("^", var), "", 
                names(hazard_ratios))
        }
        df <- data.frame(rbind(hazard_ratios))
        df$Model <- as.character(formula)
        df$N <- cox_models[[i]]$n
        df$C <- paste(signif(cox_models[[i]]$concordance[[6]],3)," (",signif(cox_models[[i]]$concordance[[7]],2),")", sep="")
        result_list[[i]] <- df
    }
    results_df <- do.call(rbind, result_list)
    results_df$Low <- "Reference"
    results_df$Model <- NULL
    return(results_df[,c("N","Low","Intermediate","High", "C")])
}

In [221]:
# Add events
result <- data.filt.df %>% 
    filter(!is.na(eval(fna3_cluster_n))) %>% 
    group_by(fna3_cluster_n) %>% 
    summarize(Events = sum(status)) %>%
    column_to_rownames(var = "fna3_cluster_n")

result

,Events
,<int>
Low,381
Intermediate,378
High,188


In [222]:
cox_models <- list(mod.dara_treated, mod.pi_treated, mod.dara_or_pi_treated, mod.firstline)
df <- coxSummary2(cox_models, "fna3_cluster_n", filt=T)
df

,N,Low,Intermediate,High,C
,<int>,<chr>,<chr>,<chr>,<chr>
hazard_ratios,282,Reference,1.65 (0.976-2.79),3.35 (1.91-5.88),0.692 (0.04)
hazard_ratios1,643,Reference,1.4 (1.08-1.82),2.01 (1.49-2.72),0.664 (0.019)
hazard_ratios2,728,Reference,1.39 (1.08-1.79),1.97 (1.48-2.64),0.666 (0.019)
hazard_ratios3,1478,Reference,1.26 (1.08-1.45),1.95 (1.63-2.34),0.647 (0.011)


In [127]:
cox_models <- list(mod.dara_treated, mod.pi_treated, mod.dara_or_pi_treated, mod.firstline)
cox_summary <- coxSummary2(cox_models, "fna3_cluster_n")
cox_summary$Low <- "Reference"
cox_summary

# Add events
result <- data.df %>% 
    filter(!is.na(eval( BU.Cardiac.Stage))) %>% 
    group_by(BU.Cardiac.Stage) %>% 
    summarize(Events = sum(status)) %>%
    column_to_rownames(var = "BU.Cardiac.Stage")

cox_summary <- rbind(data.frame(t(result)), cox_summary)
#write.table(cox_summary, "Tables/cox_mortality_hr.tsv", sep="\t")
cox_summary

,Intermediate,High,Model,Low
,<chr>,<chr>,<chr>,<chr>
hazard_ratios,1.65 (0.976-2.79),3.35 (1.91-5.88),"Surv(time_from_admission, status) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era)",Reference
hazard_ratios1,1.4 (1.08-1.82),2.01 (1.49-2.72),"Surv(time_from_admission, status) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era)",Reference
hazard_ratios2,1.39 (1.08-1.79),1.97 (1.48-2.64),"Surv(time_from_admission, status) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era)",Reference
hazard_ratios3,1.25 (1.07-1.45),2 (1.66-2.41),"Surv(time_from_admission, status) ~ fna3_cluster_n + first_line + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era)",Reference


## 3. ESKD

---

In [230]:
coxSummary3 <- function(cox_models, var, filt = TRUE) 
{
    i <- 1
    result_list = list()
    for (i in seq_along(cox_models)) {
        formula <- Reduce(paste, deparse(formula(cox_models[[i]])))
        hazard_ratios <- getCoef(cox_models[[i]])$HR
        names(hazard_ratios) <- rownames(getCoef(cox_models[[i]]))
        if (filt == T) {
            hazard_ratios <- hazard_ratios[names(hazard_ratios)[grepl(paste0("^", 
                var), names(hazard_ratios))]]
            names(hazard_ratios) <- sub(paste0("^", var), "", 
                names(hazard_ratios))
        }
        df <- data.frame(rbind(hazard_ratios))
        df$Model <- as.character(formula)
        df$N <- cox_models[[i]]$n
        df$C <- paste(signif(cox_models[[i]]$concordance[[6]],3)," (",signif(cox_models[[i]]$concordance[[7]],2),")", sep="")
        result_list[[i]] <- df
    }
    results_df <- do.call(rbind, result_list)
    results_df$High <- "Reference"
    results_df$Model <- NULL
    return(results_df[,c("N","High","Low","Intermediate", "C")])
}

In [231]:
mod.dara_treated <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=eskd.df[eskd.df$dara_received==T,], x=TRUE, tt=function(x,t,...) x * log(t))

mod.dara_or_pi_treated <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=eskd.df[eskd.df$dara_received==T | eskd.df$pi_received==T,], x=TRUE, tt=function(x,t,...) x * log(t))

mod.pi_treated <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=eskd.df[eskd.df$pi_received==T,], x=TRUE, tt=function(x,t,...) x * log(t))

# First-line
data.filt.df <- eskd.df[eskd.df$first_line %in% c("HDM/SCT","Melphalan-based regimen","Proteasome inhibitor-based", "Daratumumab-based","IMiD-based"),]
data.filt.df$first_line <- factor(data.filt.df$first_line, levels=c("HDM/SCT","Melphalan-based regimen","Proteasome inhibitor-based", "Daratumumab-based","IMiD-based")) 

mod.firstline_interaction <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n*first_line+ Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=data.filt.df, x=TRUE, tt=function(x,t,...) x * log(t))

mod.firstline <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + first_line + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=data.filt.df, x=TRUE, tt=function(x,t,...) x * log(t))

Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
"Loglik converged before variable  7 ; coefficient may be infinite. "
Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
"Loglik converged before variable  5,17,18 ; coefficient may be infinite. "


In [242]:
# Add events
result <- eskd.df[eskd.df$dara_received==T,] %>% 
    filter(!is.na(eval(fna3_cluster_n))) %>% 
    group_by(fna3_cluster_n) %>% 
    summarize(Events = sum(status)) %>%
    column_to_rownames(var = "fna3_cluster_n")

result

,Events
,<int>
High,25
Low,33
Intermediate,25


In [235]:
coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), 
    data=eskd.df[eskd.df$dara_received==T,], x=TRUE, tt=function(x,t,...) x * log(t))$concordance

Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
"Loglik converged before variable  7 ; coefficient may be infinite. "


concordant   discordant       tied.x       tied.y      tied.xy  concordance 
1.376000e+03 5.430000e+02 6.000000e+00 0.000000e+00 0.000000e+00 7.163636e-01 
         std 
5.039391e-02

In [233]:
cox_models <- list(mod.dara_treated, mod.pi_treated, mod.dara_or_pi_treated, mod.firstline)
df <- coxSummary3(cox_models, "fna3_cluster_n", filt=T)
df

,N,High,Low,Intermediate,C
,<int>,<chr>,<chr>,<chr>,<chr>
hazard_ratios,246,Reference,3.32 (0.75-14.7),7.94 (1.76-35.7),0.712 (0.045)
hazard_ratios1,530,Reference,4.5 (1.37-14.8),11.4 (3.5-37.3),0.743 (0.027)
hazard_ratios2,604,Reference,4.85 (1.48-15.8),12.5 (3.87-40.6),0.743 (0.026)
hazard_ratios3,1304,Reference,3.5 (1.6-7.63),7.91 (3.66-17.1),0.698 (0.02)
